In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pickle
import numpy as np

USE_SUBLABEL = False
URL_PER_SITE = 10
TOTAL_URLS   = 950

# Load the pickle file
print("Loading datafile...")
with open("/content/drive/MyDrive/mon_standard.pkl", 'rb') as fi: #Colab 에서 mon_standard.pkl 파일의 경로
    data = pickle.load(fi)

X8 = []  # rank 4.Standard deviation of the outgoing packet ordering list
X9 = []  # rank 5.Number of outgoing packets

y = []  # Array to store the site of each instance - 19,000 instances, e.g., [0, 0, 0, 0, 0, 0, ..., 94, 94, 94, 94, 94]

# Differentiate instances and sites, and store them in the respective x and y arrays
# x array (direction*timestamp), y array (site label)
for i in range(TOTAL_URLS):
    if USE_SUBLABEL:
        label = i
    else:
        label = i // URL_PER_SITE # Calculate which site's URL the current URL being processed belongs to and set that value as the label. Thus, URLs fetched from the same site are labeled identically.

    for sample in data[i]:

        incoming_count = 0 #X5
        outgoing_count = 0 #X9

        outgoing_packet_order = [] #X8

        last_sign = None

        for c in sample:

            # Calculate the direction and size for packet size
            dr = 1 if c > 0 else -1
            packet_size = dr * 512


           # X3
           # Calculate the burst sequence based on the sign of the value
            if c > 0:
                sign = 1
                outgoing_count += 1 #X9 (rank 5)
                outgoing_packet_order.append(abs(c))

            else:
                sign = -1
                incoming_count += 1 #X5 (rank 1)
                incoming_packet_order.append(c)



            last_sign = sign


        #X8 (rank 4)
        std_dev_outgoing_order = np.std(outgoing_packet_order)

        total_packets = len(sample)


        X8.append(std_dev_outgoing_order)
        X9.append(outgoing_count)

        y.append(label)

X8_np = np.array([np.array(xi) for xi in X8])
X9_np = np.array([np.array(xi) for xi in X9])


size = len(y)

# Extract y values and remove duplicates
unique_y = list(set(y))

y_np = np.array(y)


np.savez('/content/drive/MyDrive/data_arrays_3.npz', X8=X8_np, X9=X9_np, y=y_np)

